In [1]:
import pandas as pd
import numpy as np

In [66]:
df = pd.read_excel('../00_dataset/RFM data.xlsx') 
df.head()

,品牌名称,买家昵称,付款日期,订单状态,实付金额,邮费,省份,城市,购买数量
0,数据不吹牛,叫我李2,2019-01-01 00:17:59,交易成功,186,6,上海,上海市,1
1,数据不吹牛,0cyb1992,2019-01-01 00:59:54,交易成功,145,0,广东省,广州市,1
2,数据不吹牛,萝污萌莉,2019-01-01 07:48:48,交易成功,194,8,山东省,东营市,1
3,数据不吹牛,atblovemyy,2019-01-01 09:15:49,付款以后用户退款成功，交易自动关闭,84,0,江苏省,镇江市,1
4,数据不吹牛,小星期鱼,2019-01-01 09:59:33,付款以后用户退款成功，交易自动关闭,74,0,上海,上海市,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28833 entries, 0 to 28832
Data columns (total 9 columns):
品牌名称    28833 non-null object
买家昵称    28833 non-null object
付款日期    28833 non-null datetime64[ns]
订单状态    28833 non-null object
实付金额    28833 non-null int64
邮费      28833 non-null int64
省份      28833 non-null object
城市      28832 non-null object
购买数量    28833 non-null int64
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 2.0+ MB


In [11]:
df['订单状态'].value_counts()

交易成功                 27793
付款以后用户退款成功，交易自动关闭     1040
Name: 订单状态, dtype: int64

In [14]:
df = df.loc[df['订单状态'] == '交易成功', :]
df.head()

,品牌名称,买家昵称,付款日期,订单状态,实付金额,邮费,省份,城市,购买数量
0,数据不吹牛,叫我李2,2019-01-01 00:17:59,交易成功,186,6,上海,上海市,1
1,数据不吹牛,0cyb1992,2019-01-01 00:59:54,交易成功,145,0,广东省,广州市,1
2,数据不吹牛,萝污萌莉,2019-01-01 07:48:48,交易成功,194,8,山东省,东营市,1
5,数据不吹牛,重碎叠,2019-01-01 10:00:07,交易成功,197,0,江苏省,南京市,1
6,数据不吹牛,iho_jann,2019-01-01 10:00:08,交易成功,168,0,广东省,广州市,1


In [38]:
# 构造R/F/M值
# 构造R
df_r = df[['买家昵称', '付款日期', '实付金额']]
r = df_r.groupby('买家昵称')['付款日期'].max().reset_index()
r['R'] = (pd.to_datetime('2019-07-30') - r['付款日期']).dt.days
r = r[['买家昵称', 'R']]

# 构造F
# 引入日期标签作为辅助
df['日期标签'] = df['付款日期'].astype(str).str[:10]
dup_f = df.groupby(['买家昵称', '日期标签'])['付款日期'].count().reset_index()

f = dup_f.groupby('买家昵称')['付款日期'].count().reset_index()
f.columns = ['买家昵称', 'F']

# 构造M
sum_m = df.groupby('买家昵称')['实付金额'].sum().reset_index()
sum_m.columns = ['买家昵称', '总支付金额']
combine_fm = pd.merge(sum_m, f, left_on='买家昵称', right_on='买家昵称', how='inner')
combine_fm['M'] = combine_fm['总支付金额'] / combine_fm['F']

# 合并RFM
rfm = pd.merge(r, combine_fm, left_on='买家昵称', right_on='买家昵称', how='inner')
rfm = rfm[['买家昵称', 'R', 'F', 'M']]

# rfm['R'].describe()
# rfm['F'].describe()
# rfm['M'].describe()

# 计算rfm的score值
rfm['R-Score'] = pd.cut(rfm['R'], bins=[0, 30, 60, 90, 120, 10000], labels=[5, 4, 3, 2, 1], right=False).astype(float)
rfm['F-Score'] = pd.cut(rfm['F'], bins=[1, 2, 3, 4, 5, 10000], labels=[1, 2, 3, 4, 5], right=False).astype(float)
rfm['M-Score'] = pd.cut(rfm['M'], bins=[0, 50, 100, 150, 200, 100000], labels=[1,2, 3, 4, 5], right=False).astype(float)

# 将rfm的分数值与各自平均值作对比，对客户进行分层，将客户分为8个类别
rfm['R是否大于均值'] = (rfm['R-Score'] > rfm['R-Score'].mean()) * 1
rfm['F是否大于均值'] = (rfm['F-Score'] > rfm['F-Score'].mean()) * 1
rfm['M是否大于均值'] = (rfm['M-Score'] > rfm['M-Score'].mean()) * 1

# 构建合并指标，为客户类别打标签
rfm['客户数值'] = rfm['R是否大于均值'] * 100 + rfm['F是否大于均值'] * 10 + rfm['M是否大于均值'] * 1

def transform_label(x):
    if x == 111:
        label = '重要价值客户'
    elif x == 110:
        label = '消费潜力客户'
    elif x == 101:
        label = '频次深耕客户'
    elif x == 100:
        label = '新客户'
    elif x == 11:
        label = '重要价值流失预警客户'
    elif x == 10:
        label = '一般客户'
    elif x == 1:
        label = '高消费唤回客户'
    elif x == 0:
        label = '流失客户'
    return label

rfm['客户类型'] = rfm['客户数值'].apply(transform_label)
rfm.head()

,买家昵称,R,F,M,R-Score,F-Score,M-Score,R是否大于均值,F是否大于均值,M是否大于均值,客户数值,客户类型
0,.blue_ram,175,1,49.0,1.0,1.0,1.0,0,0,0,0,流失客户
1,.christiny,181,1,183.0,1.0,1.0,4.0,0,0,1,1,高消费唤回客户
2,.willn1,199,1,34.0,1.0,1.0,1.0,0,0,0,0,流失客户
3,.托托m,199,1,37.0,1.0,1.0,1.0,0,0,0,0,流失客户
4,0000妮,31,1,164.0,4.0,1.0,4.0,1,0,1,101,频次深耕客户


In [53]:
# 客户类别统计
label_count = rfm['客户类型'].value_counts().reset_index()
label_count.columns = ['客户类型', '人数']
label_count['人数占比'] = label_count['人数'] / label_count['人数'].sum()

rfm['付款总金额'] = rfm['F'] * rfm['M']
rfm_money = rfm.groupby('客户类型')['付款总金额'].sum().reset_index()
rfm_money.columns = ['客户类型', '消费金额']
rfm_money['金额占比'] = rfm_money['消费金额'] / rfm_money['消费金额'].sum()

label_result = label_count.merge(rfm_money)
writer = pd.ExcelWriter('RFM_Analysis_Result.xlsx')
label_result.to_excel(excel_writer=writer, sheet_name='RFM-result', index=0)
writer.save()
writer.close()

In [61]:
df.head()

,品牌名称,买家昵称,付款日期_x,订单状态,实付金额,邮费,省份,城市,购买数量,日期标签_x,付款日期_y,日期标签_y
0,数据不吹牛,叫我李2,2019-01-01 00:17:59,交易成功,186,6,上海,上海市,1,2019-01-01,2019-01-01 00:17:59,2019-01-01
1,数据不吹牛,0cyb1992,2019-01-01 00:59:54,交易成功,145,0,广东省,广州市,1,2019-01-01,2019-01-01 00:59:54,2019-01-01
2,数据不吹牛,萝污萌莉,2019-01-01 07:48:48,交易成功,194,8,山东省,东营市,1,2019-01-01,2019-01-01 07:48:48,2019-01-01
3,数据不吹牛,重碎叠,2019-01-01 10:00:07,交易成功,197,0,江苏省,南京市,1,2019-01-01,2019-01-01 10:00:07,2019-01-01
4,数据不吹牛,重碎叠,2019-01-01 10:00:07,交易成功,197,0,江苏省,南京市,1,2019-01-01,2019-01-11 23:19:23,2019-01-11


In [67]:
df_date = df.copy()
df_date.loc[:, '日期标签'] = df_date.loc[:, '付款日期'].astype(str).str[:10]
df_date = df_date.set_index('日期标签')
df['日期标签'] = df_date['日期标签']
df.head()

KeyError: '日期标签'